# Generate list of abstracts to upload

[Installation instructions](#installing-code)

Note that before running your script, you should create a copy of the submissions file called ``selected.csv`` with a new column ``decision`` which should be ``Talk``, ``Flash talk`` or ``Poster``. You probably want to use the outcome of the approval voting to fill this in, but you might then need to modify it because sometimes people will cancel etc., so we store the final version in a csv.

In [1]:
import csv
from collections import defaultdict

import jinja2

from IPython.core.display import HTML
import numpy as np
import matplotlib.pyplot as plt

from random import shuffle

This function generates a "stub" or short file name based on the title and author.

In [2]:
def generate_stub(sub):
    title_words = sub['Presentation title'].split(' ')
    title_words = [word for word in title_words if len(word)>3]
    name = ' '.join(sub['Corresponding author name'].split(' ')[:2])
    stub = name+' '+title_words[0]
    stub = stub.lower().replace(' ', '-').replace(':', '').replace('.', '').replace(',', '')
    return stub

Load in the selected talks and clean the data.

In [9]:
with open('selected.csv', newline='', encoding='utf8') as csvfile:
    csvreader = csv.DictReader(csvfile, dialect='excel')
    submissions = list(csvreader)
submissions_by_title = {}
for submission in submissions:
    submission['ID'] = submission['\ufeffID']
    for k, v in list(submission.items()):
        k_new = k.replace('\ufeff', '').strip().split('\n')[0]
        submission[k_new] = v
    submissions_by_title[generate_stub(submission)] = submission
    
shuffle(submissions) # randomize the order so that webpage order doesn't give away the voting order

As in previous notebooks, use Jinja templating to generate the list of all abstracts etc. and save them as markdown files.

In [ ]:
template = jinja2.Template('''# {{ sub['Presentation title'] }}

**Authors:** {{ sub['Presentation authors'] }}
                           
{% if sub['Presenting author name'] %}**Presenting author:** {{ sub['Presenting author name'] }}{% endif %}

**Presentation type:** {{ sub['decision'] }} at [SNUFA 2024 online workshop (5-6 Nov 2024)](https://snufa.net/2024)

## Abstract

{{ sub['Abstract (please keep under 300 words)'] }}
''')

for sub in submissions:
    stubname = generate_stub(sub)
    sub['abstract_filename'] = fname = f'abstracts/{stubname}.md'
    with open(fname, 'w', encoding='utf-8') as f:
        f.write(template.render(sub=sub))

template = jinja2.Template('''# SNUFA 2024 Abstracts

[Click here for the full programme](https://snufa.net/2024)

## Invited talks (TBD)

{% for talktype in ['Talk', 'Flash talk', 'Poster'] %}

## {{ talktype }}

{% for sub in submissions | sort(attribute='abstract_filename') %}{% if sub['decision']==talktype %}* [{{ sub['Presentation title'] }}]({{ sub['abstract_filename'] }}) ({{ sub['Presentation authors'] }})
{% endif %}{% endfor %}

{% endfor %}

''')

open('all_abstracts.md', 'w', encoding='utf-8').write(template.render(submissions=submissions))